# Notebook for testing with real data

## We will use Y@N dataset

In [1]:
import pandas as pd
import gmaps
from wenet_models import LocationPoint
from wenet_algo import estimate_stay_points, estimate_stay_regions
from datetime import datetime

In [2]:
df = pd.read_csv('/idiap/temp/wdroz/locations/eb538e9c-7297-4460-881f-0c1f46f414cb_location.csv')

In [3]:
df.head()

,userid,night,type,timestamp,timezone,local_time,source,latitude,longitude,speed,accuracy,provider,bearing
0,eb538e9c-7297-4460-881f-0c1f46f414cb,20140926,Location,1411758055,7200,20140926210055,355026050270939,46.514609,6.690031,1.118034,15,gps,24.600000
1,eb538e9c-7297-4460-881f-0c1f46f414cb,20140926,Location,1411758060,7200,20140926210100,355026050270939,46.514790,6.689949,1.250000,22,gps,233.300003
2,eb538e9c-7297-4460-881f-0c1f46f414cb,20140926,Location,1411758222,7200,20140926210342,355026050270939,46.515076,6.690051,0.250000,42,gps,337.399994
3,eb538e9c-7297-4460-881f-0c1f46f414cb,20140926,Location,1411758243,7200,20140926210403,355026050270939,46.514730,6.690049,0.000000,25,gps,0.000000
4,eb538e9c-7297-4460-881f-0c1f46f414cb,20140926,Location,1411758248,7200,20140926210408,355026050270939,46.514779,6.690355,1.750000,30,gps,213.600006


In [4]:
locations = []
for index, row in df.iterrows():
    pts_t = datetime.fromtimestamp(row['timestamp'])
    location = LocationPoint(pts_t, row['latitude'], row['longitude'])
    locations.append(location)

In [5]:
len(locations)

6178

In [36]:
stay_points = estimate_stay_points(locations, time_min_ms=5*60*100)
len(stay_points)

143

In [37]:
stay_regions = estimate_stay_regions(stay_points)
stay_regions

In [38]:
all_stay_regions_center = [[s._lat, s._lng] for s in stay_regions]
all_stay_regions_center

[[46.51472778899999, 6.69023280425],
 [46.568862505931385, 6.562881159117647],
 [46.80164280083333, 7.151114350833333]]

In [44]:
all_stay_regions_rect = [gmaps.Polygon([(s._topleft_lat, s._topleft_lng), (s._topleft_lat, s._bottomright_lng), (s._bottomright_lat, s._bottomright_lng), (s._bottomright_lat, s._topleft_lng)], fill_color='#800080') for s in stay_regions]

In [45]:
len(all_stay_regions_rect)

3

In [46]:
with open('google_api_key.txt', 'r') as f:
    key = f.read()
gmaps.configure(api_key=key)

In [47]:
raw_locations_df = df[['latitude', 'longitude']]
all_stay_regions_center_layer = gmaps.marker_layer(all_stay_regions_center)
all_stay_regions_rect_layer = gmaps.drawing_layer(features=all_stay_regions_rect)
stay_points_layer = gmaps.symbol_layer([[p._lat, p._lng] for p in stay_points], fill_color='#00FFFF', stroke_color='#00FFFF', scale=2)
locations_layer = gmaps.heatmap_layer(raw_locations_df)
fig = gmaps.figure()
fig.add_layer(stay_points_layer)
fig.add_layer(all_stay_regions_center_layer)
fig.add_layer(all_stay_regions_rect_layer)
fig.add_layer(locations_layer)
locations_layer.max_intensity = 100
locations_layer.point_radius = 10
fig.map_type = 'SATELLITE'
fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
locations_layer.max_intensity = 100
locations_layer.point_radius = 10

In [ ]:
fig.map_type = 'SATELLITE'